<a href="https://colab.research.google.com/github/malowana/NLP_budzet_obywatelski/blob/main/BO_analiza_morfologiczna2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U spacy

In [2]:
!pip install morfeusz2

In [3]:
# instalacja morfeusza
!wget -O - http://download.sgjp.pl/apt/sgjp.gpg.key|sudo apt-key add -
!sudo apt-add-repository http://download.sgjp.pl/apt/ubuntu
!sudo apt update
!sudo apt install morfeusz2
!sudo apt install python3-morfeusz2


# instalacja modelu IPI PAN dla języka polskiego
!wget "http://zil.ipipan.waw.pl/SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.3.tar.gz"
!mv 'SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.3.tar.gz' pl_spacy_model_morfeusz-0.1.3.tar.gz
!python3 -m pip install pl_spacy_model_morfeusz-0.1.3.tar.gz

# linkowanie modelu do spaCy
!python3 -m spacy link pl_spacy_model_morfeusz pl_spacy_model_morfeusz -f

--2022-01-20 19:04:23--  http://download.sgjp.pl/apt/sgjp.gpg.key
Resolving download.sgjp.pl (download.sgjp.pl)... 193.0.67.154
Connecting to download.sgjp.pl (download.sgjp.pl)|193.0.67.154|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1029 (1.0K) [application/pgp-keys]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.00K  --.-KB/s    in 0s      

2022-01-20 19:04:23 (148 MB/s) - written to stdout [1029/1029]

OK
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hi

In [4]:
#spacy.load('pl_spacy_model_morfeusz')

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from gensim.utils import simple_preprocess
from collections import Counter
import gc
import morfeusz2
import spacy
from spacy.lang.pl.stop_words import STOP_WORDS

In [6]:
morf = morfeusz2.Morfeusz()

In [7]:
for analysis in morf.analyse("sadzimy rośliny zimozielone budowa dużego skateparkuu na gocławiu dla całej pragi-południe jako wizytówka prawej strony wisły domki dla jeży nad balatonem nie chcesz mieć szczurów"):
  # na wyjściu z morf.analyse dostajemy listę interpretacji, niektóre tokeny mogą mieć więcej niż jedną interpretację, jak tu z tokenem "życia"
  start_ind, end_ind, interp = analysis
  form, lemma, tag, qualifiers, _ = interp
  try:
    pos, feats = tag.split(":", 1)
  except ValueError: # podana forma nie ma cech morfosyntaktycznych
    pos, feats = tag, ""
  print("{0:4} {1:4} {2:8} {3:8} {4:6} {5:20} {6:15}".format(start_ind, end_ind, form, lemma, pos, feats, str(qualifiers)))

   0    1 sadzimy  sadzić   fin    pl:pri:imperf        []             
   1    2 rośliny  roślina  subst  sg:gen:f             ['nazwa_pospolita']
   1    2 rośliny  roślina  subst  pl:nom.acc.voc:f     ['nazwa_pospolita']
   2    3 zimozielone zimozielony adj    pl:acc:m2.m3.f.n:pos []             
   2    3 zimozielone zimozielony adj    pl:nom.voc:m2.m3.f.n:pos []             
   2    3 zimozielone zimozielony adj    sg:acc:n:pos         []             
   2    3 zimozielone zimozielony adj    sg:nom.voc:n:pos     []             
   3    4 budowa   budowa   subst  sg:nom:f             ['nazwa_pospolita']
   4    5 dużego   duży     adj    sg:acc:m1.m2:pos     []             
   4    5 dużego   duży     adj    sg:gen:m1.m2.m3.n:pos []             
   5    6 skateparkuu skateparkuu ign                         []             
   6    7 na       na:I     interj                      []             
   6    7 na       na:P     prep   acc                  []             
   6    7 na     

In [8]:
df = pd.read_excel('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/budzet_obywatelski.xlsx', sheet_name='Arkusz1')
df.rename(columns={'rok ': 'rok', 'miasto ':'miasto'}, inplace=True)
df = df[~df['nazwa'].isna()]
df['miasto'] = df['miasto'].map(lambda x: x.upper())
# Usunięcie pustych wpisów

# Indeksy do usunięcia
rem_index = df[df['nazwa'].isnull()].index.tolist()
rem_index += df[df['nazwa']=='bd'].index.tolist()

df.drop(df.index[rem_index], inplace=True)
# Usunięcie duplikatów
df = df.drop_duplicates()
df['nazwa'] = df['nazwa'].apply(str)
#usuniecie kolumny wszystko
if 'WSZYSTKO' in df: del df['WSZYSTKO']
df.sample(10)

,rok,miasto,nazwa
177,2020,GDYNIA,"Warsztaty wspomagające koncentrację, pamięć i ..."
2238,2018,BIELSKO-BIAŁA,Wykonanie miejsc parkingowych w tym dla osób z...
9239,2019,WROCŁAW,Rekreacyjny Szlak „Dolina Odry” - etap I Bisku...
5531,2020,TYCHY,Plac do gry w bule w Parku Suble
5687,2018,ZABRZE,Świetlica na błysk ― wymiana konstrukcji podło...
2758,2018,CZĘSTOCHOWA,Bezplatne Kino na Lezakach W Czestochowie
20639,2019,WARSZAWA,„Mini Poradnia” na wschodniej Białołęce – bezp...
5184,2020,ŚWIĘTOCHŁOWICE,9. Eksperymentuj na świeżym powietrzu!
5213,2018,TYCHY,Mikołajki na Czułowie
12037,2020,ZIELONA GÓRA,bd


In [9]:
df.loc[(df['miasto'] == 'WARSZAWA'), "czy_WAW"] = 1
df.loc[df["czy_WAW"].isnull(),  "czy_WAW"] = 0
df["czy_WAW"].value_counts()

0.0    18431
1.0     4599
Name: czy_WAW, dtype: int64

# WARSZAWA

In [10]:
df_waw = df[df['miasto'] == 'WARSZAWA']
df_waw.head()

,rok,miasto,nazwa,czy_WAW
18418,2018,WARSZAWA,Domki dla jeży,1.0
18419,2018,WARSZAWA,Wolskie lato filmowe,1.0
18420,2018,WARSZAWA,Chronimy jerzyki i wróble na Woli - skrzynki l...,1.0
18421,2018,WARSZAWA,Czysta Wisła na Pradze Południe,1.0
18422,2018,WARSZAWA,Wola na 2 koła! Kontraruch rowerowy w całej dz...,1.0


In [11]:
#zlaczenie wszystkich nazw w jeden string
#df_waw['nazwa'] = df_waw['nazwa'].apply(str)
docs_waw = ' '.join(df_waw.nazwa)
docs_waw = docs_waw.lower()
docs_waw

'domki dla jeży wolskie lato filmowe chronimy jerzyki i wróble na woli - skrzynki lęgowe czysta wisła na pradze południe wola na 2 koła! kontraruch rowerowy w całej dzielnicy bielańska plaża nad wisłą chronimy jerzyki i wróble na pradze -południe - skrzynki lęgowe leczenie i sterylizacja wolno żyjących kotów ćwiczenia dla seniorów na otwartych siłowniach rozwijamy wolskie maluchy skarby woli. czego nie wiesz o swojej dzielnicy? chronimy jerzyki i wróble na bielanach - skrzynki lęgowe czyste powietrze dla targówka - nasadzenia drzew tai chi na świeżym powietrzu ogródek przy przystanku autobusowym "antka rozpylacza 01" wiem czym oddycham! - tablica informująca o poziomie zanieczyszczenia powietrza (praga-południe) virtualna warszawa – turystyka dostępna dla wszystkich, również niepełnosprawnych. ratujemy drzewa parku skaryszewskiego - wiązania elastyczne w koronach drzew na podstawie ekspertyzy dendrologicznej. serek - miejsce spotkań i relaksu przy metrze słodowiec wodny plac zabaw dla 

In [12]:
for analysis in morf.analyse(docs_waw):
  # na wyjściu z morf.analyse dostajemy listę interpretacji, niektóre tokeny mogą mieć więcej niż jedną interpretację, jak tu z tokenem "życia"
  start_ind, end_ind, interp = analysis
  form, lemma, tag, qualifiers, _ = interp
 
  attribs = ['start_ind', 'end_ind', 'form', 'lemma', 'pos', 'feats', 'str(qualifiers)']
  table = [{att:tok.__getattribute__(att) for att in attribs} for tok in analysis2]
  df_waw_morf = pd.DataFrame(table)
  print(df_waw_morf[attribs])

NameError: ignored

In [ ]:
attribs = ['orth_', 'lemma_', 'tag_', 'pos_', 'dep_', 'head', 'is_stop', 'is_punct', 'is_digit']
table = [{att:tok.__getattribute__(att) for att in attribs} for tok in doc]
df_waw_morf = pd.DataFrame(table)
print(df_waw_morf[attribs])

In [18]:
df_waw2 = df_waw.sample(5)
df_waw2.head()

,rok,miasto,nazwa,czy_WAW
21157,2019,WARSZAWA,Klub złotej rybki,1.0
20440,2019,WARSZAWA,Rower to mój dziecięcy świat! — utworzenie now...,1.0
20137,2018,WARSZAWA,Rowerowe połączenie Placu Piłsudskiego z Krako...,1.0
21136,2019,WARSZAWA,Kolorowy wybieg dla wszystkich w SP 86,1.0
19433,2018,WARSZAWA,Lato z dyskoteką nad Balatonem,1.0


In [95]:
df_waw_morf = pd.DataFrame()
data4 =[]
i = 0

In [ ]:
for rows in df_waw2.rows:
  for interpretation for df_waw2.nazwa[rows]

In [96]:
for text in df_waw.nazwa:
    print(text)
    analysis = morf.analyse(text)
    i=0
    for interpretation in analysis:
      start_ind, end_ind, interp = interpretation
      form, lemma, tag, qualifiers, _ = interpretation[2]
      data4.append(('Warszawa', text, start_ind, end_ind, form,lemma,tag,qualifiers,_))

Domki dla jeży
Wolskie lato filmowe
Chronimy jerzyki i wróble na Woli - skrzynki lęgowe
Czysta Wisła na Pradze Południe
Wola na 2 koła! Kontraruch rowerowy w całej dzielnicy
Bielańska plaża nad Wisłą
Chronimy jerzyki i wróble na Pradze -Południe - skrzynki lęgowe
leczenie i sterylizacja wolno żyjących kotów
Ćwiczenia dla seniorów na otwartych siłowniach
Rozwijamy wolskie maluchy
Skarby Woli. Czego nie wiesz o swojej dzielnicy?
Chronimy jerzyki i wróble na Bielanach - skrzynki lęgowe
Czyste powietrze dla Targówka - nasadzenia drzew
Tai Chi na świeżym powietrzu
Ogródek przy przystanku autobusowym "Antka Rozpylacza 01"
Wiem czym oddycham! - tablica informująca o poziomie zanieczyszczenia powietrza (Praga-Południe)
Virtualna Warszawa – turystyka dostępna dla wszystkich, również niepełnosprawnych.
Ratujemy drzewa Parku Skaryszewskiego - wiązania elastyczne w koronach drzew na podstawie ekspertyzy dendrologicznej.
Serek - Miejsce spotkań i relaksu przy metrze Słodowiec
Wodny plac zabaw dla B

In [99]:
cols=['miasto','nazwa','start_ind', 'end_ind', 'form','lemma','tag','qualifiers','_']
df_waw_morf4 = pd.DataFrame(data4, columns=cols)
df_waw_morf4.head(40)

,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,Warszawa,Domki dla jeży,0,1,Domki,Domka:Sf,subst:sg:gen:f,"[imię, nazwisko]",[]
1,Warszawa,Domki dla jeży,0,1,Domki,Domka:Sf,subst:pl:nom.acc.voc:f,"[imię, nazwisko]",[]
2,Warszawa,Domki dla jeży,0,1,Domki,Domka:Sm1,subst:sg:gen:m1,[nazwisko],[]
3,Warszawa,Domki dla jeży,0,1,Domki,domek,subst:pl:nom.acc.voc:m3,[nazwa_pospolita],[]
4,Warszawa,Domki dla jeży,0,1,Domki,Domka:Sm1,depr:pl:nom.acc.voc:m2,[nazwisko],[]
5,Warszawa,Domki dla jeży,0,1,Domki,Domek:Sm1,depr:pl:nom.acc.voc:m2,"[imię, nazwisko]",[]
6,Warszawa,Domki dla jeży,1,2,dla,dla,prep:gen,[],[]
7,Warszawa,Domki dla jeży,2,3,jeży,jeż:Sm2,subst:pl:gen:m2,[nazwa_pospolita],[zool.]
8,Warszawa,Domki dla jeży,2,3,jeży,jeż:Sm3,subst:pl:gen:m3,[nazwa_pospolita],[]
9,Warszawa,Domki dla jeży,2,3,jeży,jeżyć,fin:sg:ter:imperf,[],[]


In [100]:
df_waw_morf4.tag.value_counts()

interp                                       5123
subst:sg:gen:f                               3220
subst:sg:nom.acc:m3                          3107
subst:sg:nom:f                               2852
interj                                       2842
                                             ... 
ppron3:sg:loc:f:ter:akc.nakc:praep.npraep       1
adj:pl:nom.voc:m2.m3.f.n:sup                    1
ppron3:sg:gen:n:ter:nakc:npraep                 1
ppron3:sg:dat:m1.m2.m3:ter:nakc:npraep          1
bedzie:sg:pri:imperf                            1
Name: tag, Length: 341, dtype: int64

In [103]:
df_waw_morf4.tag.str.contains('bedzie').head()

0    False
1    False
2    False
3    False
4    False
Name: tag, dtype: bool

In [107]:
df_waw_morf4[df_waw_morf4['tag'].str.contains('inf', na=False)].head()

,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
943,Warszawa,Doświetlenie przejść dla pieszych na Ursynowie...,1,2,przejść,przejść,inf:perf,[],[]
959,Warszawa,"Ucywilizować przejście piesze ""Plażowa""",0,1,Ucywilizować,ucywilizować,inf:perf,[],[]
1316,Warszawa,"Ławki w cieniu, remont wejść i kwiaty w Parku ...",5,6,wejść,wejść,inf:perf,[],[]
1621,Warszawa,Doświetlić Międzynarodową przy Parku Skaryszew...,0,1,Doświetlić,doświetlić,inf:perf,[],[]
2192,Warszawa,Doświetlenie przejść dla pieszych na Ursynowie...,1,2,przejść,przejść,inf:perf,[],[]


In [114]:
czasownik = ['fin', 'bedzie', 'aglt', 'praet', 'impt', 'imps', 'inf', 'pcon', 'pant', 'ger', 'pact', 'ppas']
df_waw_morf4_verb = df_waw_morf4[df_waw_morf4['tag'].str.contains('|'.join(czasownik))]

In [116]:
df_waw_morf4_verb.lemma.value_counts().head(40)

zająć           365
zabawić         216
spotkać         185
przejść         148
oświetlić       127
zielenić        111
wzdłużyć        102
zakupić          99
nasadzić         93
być              61
doświetlić       46
żyć              46
skrzyżować       46
chronić          41
urządzić         35
programować      34
otworzyć         34
pić              34
szkolić          32
dojść            27
ratować          27
doposażyć        26
ćwiczyć          26
śmiecić          26
woleć            26
rozwijać         25
przystać:Vp      24
przestrzenić     23
mieć             22
parkować         21
grać             21
czytać           20
wesprzeć         19
zwalniać         19
brakować         19
wyposażyć        18
posadzić         18
zjednoczyć       18
chcieć           16
ogrodzić         16
Name: lemma, dtype: int64